### Setup Software Environment

In [1]:
from csv import DictReader
from dateutil import parser
import os
import re
import urllib2

### Download the Cincinnati 311 (Non-Emergency) Service Requests data]
- [Dataset Description](https://data.cincinnati-oh.gov/Thriving-Healthy-Neighborhoods/Cincinnati-311-Non-Emergency-Service-Requests/4cjh-bm8b/about)
- [Example of downloading a *.csv file progamatically using urllib2](http://stackoverflow.com/questions/30762150/trying-to-download-data-from-url-with-csv-file)

In [2]:
data_dir = "./Data"

csv_file_path = os.path.join(data_dir, "cincinnati311.csv")

if not os.path.exists(csv_file_path):

    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
        
    url = 'https://data.cincinnati-oh.gov/api/views' +\
          '/4cjh-bm8b/rows.csv?accessType=DOWNLOAD'

    response = urllib2.urlopen(url)

    html = response.read()

    with open(csv_file_path, 'wb') as h_file:
        h_file.write(html)

### Parse the 1st record

In [3]:
h_file = open("./Data/cincinnati311.csv", "r")
fieldnames = [re.sub("_", "", elem.lower()) for elem in h_file.readline().rstrip().split(',')]

readerobj = DictReader(h_file, fieldnames)

print readerobj.next()

h_file.close()

{'status': 'CLOS', 'servicenotice': '', 'description': '"Thank you so much for sending a plow so quickly last week when we requested one. Once plowing of main roads is finished, Colter Ave once again desperately needs help! The street is unplowed, but more problematic is the giant snow/ice pile at the entrance to the street from Sutton Avenue. It is almost impassible, and definitely dangerous.Thanks in advance for your help!"', 'statusnotes': '', 'expecteddatetime': '2014-02-06T00:00:00Z', 'lasttableupdate': '03/05/2015 11:07:49 PM +0000', 'requesteddate': '02/05/2014 12:00:00 AM +0000', 'zipcode': '45230', 'longitude': '-84.3911002774317', 'requesteddatetime': '2014-02-05T15:48:00Z', 'addressid': '', 'updateddate': '03/07/2014 12:00:00 AM +0000', 'servicename': '"Slippery streets, request"', 'servicecode': '"SLPYST"', 'address': '"6055 COLTER AV, CINC - GJ2564429820"', 'latitude': '39.0883480578611', 'updateddatetime': '2014-03-07T00:00:00Z', 'agencyresponsible': 'Public Services', 's

### Implement a class that parses and cleans a Cincinnati 311 data record
- This class forms the basis for mapper functions
- [This software applies the dateutil package parser fucntion to parse date/time strings](http://stackoverflow.com/questions/10494312/parsing-time-string-in-python)

In [4]:
class Cincinnati311CSVDataParser(object):
    """ Class that parses and cleans a Cincinnati 311 Comma Seperated Value
    (CSV) file record
    
    Data set description:
    --------------------
    https://data.cincinnati-oh.gov/Thriving-Healthy-Neighborhoods/
        Cincinnati-311-Non-Emergency-Service-Requests/4cjh-bm8b/about"""

    def __init__(self,
                 h_file):
        """ Cincinnati311CSVDataParser class constructor

        Args:
            self: Cincinnati311CSVDataParser class object handle
            
            h_file: Cincinnati 311 csv file handle
        
        Returns:
            None"""
        fieldnames = ['jurisdictionid',
                      'servicerequestid',
                      'status',
                      'statusnotes',
                      'servicename',
                      'servicecode',
                      'description',
                      'agencyresponsible',
                      'servicenotice',
                      'requesteddatetime',
                      'updateddatetime',
                      'expecteddatetime',
                      'address',
                      'addressid',
                      'zipcode',
                      'latitude',
                      'longitude',
                      'requesteddate',
                      'updateddate',
                      'lasttableupdate']
        
        matchobj = re.compile('.*date.*')

        self.date_fields = filter(lambda elem: matchobj.match(elem) != None,
                                  fieldnames)
        
        self.string_fields = filter(lambda elem: matchobj.match(elem) == None,
                                    fieldnames)
        
        self.readerobj = DictReader(h_file, fieldnames)

    def parse(self):
        """ Parses a Cincinnati 311 CSV file record
        
        Args:
            self: Cincinnati311CSVDataParser class object handle
        
        Returns:
            record: Dictionary that stores a Cincinnati 311 data CSV file
                    record"""
        record = self.readerobj.next()
        
        if record['jurisdictionid'] == 'JURISDICTION_ID':
            record = None
        else:
            
            for key in self.date_fields:
                record[key] = parser.parse(record[key])
                
            for key in self.string_fields:
                record[key] = re.sub("\"", "", record[key].lower())
        
        return record

In [5]:
h_file = open("./Data/cincinnati311.csv", "r")

parserobj = Cincinnati311CSVDataParser(h_file)

print parserobj.parse()
print parserobj.parse()

h_file.close()

None
{'status': 'clos', 'servicenotice': '', 'description': 'thank you so much for sending a plow so quickly last week when we requested one. once plowing of main roads is finished, colter ave once again desperately needs help! the street is unplowed, but more problematic is the giant snow/ice pile at the entrance to the street from sutton avenue. it is almost impassible, and definitely dangerous.thanks in advance for your help!', 'statusnotes': '', 'expecteddatetime': datetime.datetime(2014, 2, 6, 0, 0, tzinfo=tzutc()), 'lasttableupdate': datetime.datetime(2015, 3, 5, 23, 7, 49, tzinfo=tzutc()), 'requesteddate': datetime.datetime(2014, 2, 5, 0, 0, tzinfo=tzutc()), 'zipcode': '45230', 'longitude': '-84.3911002774317', 'requesteddatetime': datetime.datetime(2014, 2, 5, 15, 48, tzinfo=tzutc()), 'addressid': '', 'updateddate': datetime.datetime(2014, 3, 7, 0, 0, tzinfo=tzutc()), 'servicename': 'slippery streets, request', 'servicecode': 'slpyst', 'address': '6055 colter av, cinc - gj25644